In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.firefox.service import Service
import random
from dotenv import load_dotenv
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from selenium.common.exceptions import NoSuchElementException
import sqlite3
import os
import pandas as pd
from selenium.common.exceptions import TimeoutException

In [ ]:
def driver_setting():
    driver_path = "./geckodriver"
    options = webdriver.FirefoxOptions()
    # options.add_argument('--headless')
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")

    service = Service(executable_path=driver_path)
    driver = webdriver.Firefox(service=service, options=options)

    return driver


def sleep_time(min_sec, max_sec):
    time.sleep(random.uniform(min_sec, max_sec))
    driver.implicitly_wait(random.uniform(min_sec, max_sec))


def find_n_click_xpath(xpath, min=1.4, max=2):
    element = driver.find_element(By.XPATH, xpath)
    element.click()
    sleep_time(min, max)


def find_n_get_text_xpath(xpath):
    try:
        texted = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, xpath))).text
        if texted == "":
            texted = None
        return texted
    except TimeoutException:
        return None


def find_n_get_text_xpath_for_scores(xpath):
    try:
        texted = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text
        if texted == "":
            return "not_found"
        return texted
    except TimeoutException:
        return "not_found"


def get_href(xpath):
    element = driver.find_element(By.XPATH, xpath)
    href = element.get_attribute("href")
    return href


def everytime_login(id, pw):
    driver.get("https://account.everytime.kr/login")
    sleep_time(1, 2)

    driver.find_element(By.NAME, 'id').send_keys(id)
    driver.find_element(By.NAME, 'password').send_keys(pw)
    sleep_time(0.3, 1)

    driver.find_element(
        By.CSS_SELECTOR, '.container > form:nth-child(2) > input:nth-child(2)').click()
    sleep_time(0.5, 1)


def find_n_get_review_page(lecNum):
    driver.get('https://everytime.kr/timetable/2024/2')
    sleep_time(1, 2)

    find_n_click_xpath("/html/body/div[1]/ul/li[1]")  # 수업목록에서 검색 버튼
    find_n_click_xpath("/html/body/div[2]/div[1]/a[4]", 0.3, 0.8)  # 검색어 버튼
    find_n_click_xpath(
        "/html/body/div[1]/form[5]/div/label[3]", 0.3, 0.8)  # 과목 코드 버튼

    input_element = driver.find_element(
        By.XPATH, '/html/body/div[1]/form[5]/div/input')  # input에 클릭
    input_element.send_keys(lecNum)  # 값 넣기
    sleep_time(0.5, 1)

    find_n_click_xpath("/html/body/div[1]/form[5]/input")
    sleep_time(1, 2)

    try:
        a_path = driver.find_element_by_xpath(
            "/html/body/div[2]/div[2]/table/tbody/tr/td[7]/a")
        href = a_path.get_attribute('href')
    except NoSuchElementException:
        href = None

    return href


########################################################################


database = "./kwu-lecture-recommendation-v8.db"
conn = sqlite3.connect(database)
cursor = conn.cursor()

# 1학기
query = """
SELECT ll.lectureID, ll.lecNumber 
FROM LectureList ll
WHERE ll.semester = '2학기' 
AND ll.lectureID NOT IN (
    SELECT ld.lectureID 
    FROM LectureEverytimeData ld
    WHERE ld.crawling_date_int > 1
);
"""

cursor.execute(query)

lectureIDS = cursor.fetchall()

print(lectureIDS)
cursor.close()
conn.close()


driver = driver_setting()

load_dotenv()

everytime_login(os.getenv("etid"), os.getenv("etpw"))

for rows in lectureIDS:
    lectureID, lectureNum = rows
    print("|")
    try:
        start_time = time.time()
        print(
            f"|      | crawling (lectureID: {lectureID}, lecNum: {lectureNum}) ")
        href = find_n_get_review_page(lectureNum)
        if href == None:
            continue
        driver.get(href)

        print(f"| good | moved to {href}")

        sleep_time(1, 1.5)

        check_for_graph_page = 0
        check_for_is_null_page = 0
        try:
            check_for_graph_page = driver.find_elements(
                By.XPATH, '/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div/div[1]/div[2]')
        except:
            pass

        try:
            check_for_is_null_page = driver.find_element_by_xpath(
                '/html/body/div/div/div[2]/div/section[2]/div/span[1]')
        except:
            pass

        if len(check_for_graph_page) > 0:
            print("| info | is graph")
            star = float(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[1]/div[1]/span[1]"))

            assignment_low = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div/div[1]/div[2]").strip('%'))
            assignment_mid = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div/div[2]/div[2]").strip('%'))
            assignment_many = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div/div[3]/div[2]").strip('%'))
            assignmentAmount = (assignment_many +
                                assignment_mid*0.5 + assignment_low*0)

            teamplay_low = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[2]/div/div[1]/div[2]").strip('%'))
            teamplay_mid = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[2]/div/div[2]/div[2]").strip('%'))
            teamplay_many = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[2]/div/div[3]/div[2]").strip('%'))

            teamplayAmount = (teamplay_many +
                              teamplay_mid*0.5 + teamplay_low*0)

            grade_low = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[3]/div/div[1]/div[2]").strip('%'))
            grade_mid = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[3]/div/div[2]/div[2]").strip('%'))
            grade_many = int(find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[3]/div/div[3]/div[2]").strip('%'))

            gradeAmount = (grade_many +
                           grade_mid*0.5 + grade_low*0)

            checkAttend_str = find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[4]/div/span[1]")
            testNum_str = find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[5]/div/span[1]")

            try:
                conn = sqlite3.connect('./kwu-lecture-recommendation-v8.db')
                cursor = conn.cursor()

                lectureID = lectureID
                everytimeUrl = href
                crawling_date = 241117

                sql = '''
                INSERT OR REPLACE INTO LectureEverytimeData (lectureID, everytimeUrl, star, assignmentAmount, teamPlayAmount, gradeAmount, reviewSummary, checkAttend, testNum, crawling_date_int)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                '''

                cursor.execute(sql, (lectureID, everytimeUrl, star, assignmentAmount,
                               teamplayAmount, gradeAmount, None, checkAttend_str, testNum_str, crawling_date))
                conn.commit()
                cursor.close()
                conn.close()
                print(
                    f"| good | star: {star}, assignment: {assignmentAmount}%, teamplay: {teamplayAmount}%, grade: {gradeAmount}% \t | attend: {checkAttend_str}, test: {testNum_str}")
            except Exception as e:
                print("|  err | failed to fill? db\n", e)

        elif check_for_is_null_page:
            try:
                conn = sqlite3.connect('./kwu-lecture-recommendation-v8.db')
                cursor = conn.cursor()

                lectureID = lectureID
                everytimeUrl = href
                crawling_date = 241117

                sql = '''
                INSERT OR REPLACE INTO LectureEverytimeData (lectureID, crawling_date_int)
                VALUES (?, ?);
                '''

                cursor.execute(sql, (lectureID, crawling_date))
                conn.commit()
                cursor.close()
                conn.close()
                print(
                    f"| good | --skip--")
            except Exception as e:
                print("|  err | failed to fill? db\n", e)

        else:
            print("| info | is text")

            try:
                star = float(WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div[2]/div/section[2]/div[1]/div/span[1]"))).text)
                if star == "":
                    continue
            except TimeoutException:
                try:
                    conn = sqlite3.connect(
                        './kwu-lecture-recommendation-v8.db')
                    cursor = conn.cursor()

                    lectureID = lectureID
                    everytimeUrl = href
                    crawling_date = 241117

                    sql = '''
                    INSERT OR REPLACE INTO LectureEverytimeData (lectureID, crawling_date_int)
                    VALUES (?, ?);
                    '''

                    cursor.execute(sql, (lectureID, crawling_date))
                    conn.commit()
                    cursor.close()
                    conn.close()
                    print(
                        f"| good | --skip--")
                except Exception as e:
                    print("|  err | failed to fill? db\n", e)

            assignmentAmount_str = find_n_get_text_xpath_for_scores(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div[1]/span").strip("%")
            teamplayAmount_str = find_n_get_text_xpath_for_scores(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div[2]/span").strip("%")
            gradeAmount_str = find_n_get_text_xpath_for_scores(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div[3]/span").strip("%")

            assignment_amount_map = {
                "없음": 10,
                "보통": 50,
                "많음": 90
            }
            grade_amount_map = {
                "너그러움": 10,
                "보통": 50,
                "깐깐함": 90
            }

            assignmentAmount = assignment_amount_map.get(
                assignmentAmount_str, 0.1)
            teamplayAmount = assignment_amount_map.get(teamplayAmount_str, 0.1)
            gradeAmount = grade_amount_map.get(gradeAmount_str, 0.1)

            checkAttend_str = find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[2]/div/span")
            testNum_str = find_n_get_text_xpath(
                "/html/body/div/div/div[2]/div/section[2]/div[2]/div[3]/div/span")

            try:
                conn = sqlite3.connect('./kwu-lecture-recommendation-v8.db')
                cursor = conn.cursor()

                lectureID = lectureID
                everytimeUrl = href
                crawling_date = 241117

                sql = '''
                INSERT OR REPLACE INTO LectureEverytimeData (lectureID, everytimeUrl, star, assignmentAmount, teamPlayAmount, gradeAmount, reviewSummary, checkAttend, testNum, crawling_date_int)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                '''

                cursor.execute(sql, (lectureID, everytimeUrl, star, assignmentAmount,
                               teamplayAmount, gradeAmount, None, checkAttend_str, testNum_str, crawling_date))
                conn.commit()
                cursor.close()
                conn.close()
                print(
                    f"| good | star: {star}, assignment: {assignmentAmount}%, teamplay: {teamplayAmount}%, grade: {gradeAmount}% \t | attend: {checkAttend_str}, test: {testNum_str}")
            except Exception as e:
                print("|  err | failed to fill? db\n|      |", e)

        end_time = time.time()
        loop_time = end_time - start_time
        print(f"|      | info loop execution time: {loop_time:.2f} seconds")

    except Exception as e:
        print(f"\n!! error !! at lectureID: {lectureID[0]}\n{e}\n\n")

[(3243, '0000-1-4102-02'), (3244, '0000-1-9340-01'), (3245, '0000-1-9340-02'), (3246, '0000-1-4099-01'), (3247, '0000-1-4099-02'), (3248, '0000-1-7461-01'), (3249, '0000-1-7461-02'), (3250, '0000-1-5674-01'), (3251, '0000-1-5674-02'), (3252, '0000-1-9885-01'), (3253, '0000-1-4101-01'), (3254, '0000-1-4101-02'), (3255, '0000-1-1640-01'), (3256, '0000-1-3293-01'), (3257, '0000-1-4125-01'), (3258, '0000-1-4125-02'), (3259, '0000-2-6019-01'), (3260, '0000-2-6957-01'), (3261, '0000-2-6957-02'), (3262, '0000-2-3298-01'), (3263, '0000-2-9251-01'), (3264, '0000-2-9251-02'), (3265, '0000-2-3029-01'), (3266, '0000-2-3029-02'), (3267, '0000-2-3029-03'), (3268, '0000-2-2991-01'), (3269, '0000-2-2991-02'), (3270, '0000-2-2994-01'), (3271, '0000-3-5765-01'), (3272, '0000-3-2997-01'), (3273, '0000-3-2997-02'), (3274, '0000-4-7462-01'), (3275, '0000-1-5899-01'), (3276, '0000-1-5899-02'), (3277, '0000-1-0221-01'), (3278, '0000-1-3925-01'), (3279, '0000-1-8085-01'), (3280, '0000-1-8085-02'), (3281, '000

TypeError: 'int' object is not subscriptable